### Installing and Importing all the necessary packages

*Hey, We are Siddhant & Pavana and we are participating in this Hackathon :3. To begin our OCR-TO-JSON journey, we need to install and import the necessary libraries. This setup ensures we have all the tools required for data manipulation, visualization, and interaction with the PHI-3-Mini Model.*

In [ ]:
!pip install -U -q transformers peft==0.4.0 accelerate bitsandbytes trl

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kfp 2.5.0 requires google-cloud-storage<3,>=2.2.1, but you have google-cloud-storage 1.44.0 which is incompatible.


In [ ]:
import os
import torch
import torch.nn as nn
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline,
    logging,
)
from accelerate import PartialState
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from tqdm import tqdm
from trl import SFTConfig, SFTTrainer
from trl.trainer import ConstantLengthDataset
import json
from rich import (inspect, print, pretty)
from rich.console import Console
from rich.syntax import Syntax
from random import randrange
pretty.install()
torch.backends.cudnn.benchmark = True

2024-07-09 15:22:39.971291: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-09 15:22:39.971392: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-09 15:22:40.144294: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


### Loading and Preparing Data

*We load and prepare the dataset for fine-tuning our model. This involves splitting the dataset into training and evaluation subsets, ensuring a representative distribution for effective model training and validation.*

In [ ]:
dataset = load_dataset("mychen76/invoices-and-receipts_ocr_v1")
new_model = "elucidator8918/phi_miniocr_to_json"
dataset

Generating train split:   0%|          | 0/2043 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/125 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/70 [00:00<?, ? examples/s]


DatasetDict({
    train: Dataset({
        features: ['image', 'id', 'parsed_data', 'raw_data'],
        num_rows: 2043
    })
    test: Dataset({
        features: ['image', 'id', 'parsed_data', 'raw_data'],
        num_rows: 125
    })
    valid: Dataset({
        features: ['image', 'id', 'parsed_data', 'raw_data'],
        num_rows: 70
    })
})

In [ ]:
print(f"dataset size: {len(dataset['train'])}")
sample=dataset['train'][randrange(len(dataset))]
ocr_words=eval(sample['raw_data'])['ocr_words']
ocr_boxes=eval(sample['raw_data'])['ocr_boxes']
ocr_json=eval(sample['parsed_data'])['json']

dataset size: 2043

### Alpaca Prompt Template Definition

*The alpaca_prompt template is defined to structure input for the Phi model fine-tuning. It sets the context for generating responses based on the instructional context given.*

In [ ]:
def format_train_instruction(sample):
    return f""" <|user|>
### Instruction:
You are an expert in POS receipts and a receipt data engineer with many years of experience working with complex receipt structures.
Your task is to parse, detect, recognize, and convert the following receipt OCR image result into a structured receipt format.
The output must be a well-formed JSON object.

### Input:
{eval(sample['raw_data'])['ocr_boxes']}
<|end|>

<|system|>
{eval(sample['parsed_data'])['json']}
<|end|>
"""

In [ ]:
from random import randrange
print(format_train_instruction(dataset["train"][randrange(len(dataset))]))

<|user|> 
### Instruction:
You are an expert in POS receipts and a receipt data engineer with many years of experience working with complex 
receipt structures. 
Your task is to parse, detect, recognize, and convert the following receipt OCR image result into a structured 
receipt format. 
The output must be a well-formed JSON object.

### Input:
[[[[196.0, 110.0], [801.0, 110.0], [801.0, 161.0], [196.0, 161.0]], ('Invoice no: 39280409', 0.9981233477592468)], 
[[[196.0, 212.0], [521.0, 212.0], [521.0, 259.0], [196.0, 259.0]], ('Date of issue:', 0.9986030459403992)], 
[[[1204.0, 216.0], [1469.0, 216.0], [1469.0, 256.0], [1204.0, 256.0]], ('07/06/2014', 0.9997870326042175)], 
[[[192.0, 665.0], [388.0, 665.0], [388.0, 716.0], [192.0, 716.0]], ('Seller:', 0.9997318387031555)], [[[1240.0, 
661.0], [1429.0, 661.0], [1429.0, 720.0], [1240.0, 720.0]], ('Client:', 0.9994950890541077)], [[[207.0, 756.0], 
[425.0, 756.0], [425.0, 808.0], [207.0, 808.0]], ('Davis PLC', 0.9979212880134583)], [[[1245.0, 752.0], [1525.0, 
757.0], [1524.0, 808.0], [1244.0, 804.0]], ('Bailey-Harris', 0.9995162487030029)], [[[210.0, 811.0], [594.0, 
811.0], [594.0, 859.0], [210.0, 859.0]], ('72057 Castillo Via', 0.9990554451942444)], [[[1248.0, 811.0], [1650.0, 
811.0], [1650.0, 859.0], [1248.0, 859.0]], ('106 Michael Street', 0.9975928068161011)], [[[210.0, 866.0], [683.0, 
866.0], [683.0, 914.0], [210.0, 914.0]], ('Deniseshire, KY 95233', 0.9976860880851746)], [[[1248.0, 866.0], 
[1772.0, 866.0], [1772.0, 914.0], [1248.0, 914.0]], ('North William, DC 96680', 0.981005072593689)], [[[207.0, 
972.0], [639.0, 972.0], [639.0, 1020.0], [207.0, 1020.0]], ('Tax Id: 938-79-9168', 0.9895579814910889)], [[[1248.0,
972.0], [1676.0, 972.0], [1676.0, 1020.0], [1248.0, 1020.0]], ('Tax Id: 965-83-8071', 0.9874200224876404)], 
[[[203.0, 1019.0], [945.0, 1023.0], [945.0, 1074.0], [203.0, 1071.0]], ('IBAN: GB39WJUU70698169375316', 
0.9955376386642456)], [[[188.0, 1140.0], [377.0, 1140.0], [377.0, 1191.0], [188.0, 1191.0]], ('ITEMS', 
0.973893940448761)], [[[233.0, 1268.0], [303.0, 1268.0], [303.0, 1308.0], [233.0, 1308.0]], ('No.', 
0.9999094009399414)], [[[358.0, 1268.0], [576.0, 1268.0], [576.0, 1308.0], [358.0, 1308.0]], ('Description', 
0.9997653961181641)], [[[1012.0, 1261.0], [1097.0, 1261.0], [1097.0, 1316.0], [1012.0, 1316.0]], ('Qty', 
0.9998303055763245)], [[[1159.0, 1264.0], [1237.0, 1264.0], [1237.0, 1308.0], [1159.0, 1308.0]], ('UM', 
0.9347040057182312)], [[[1348.0, 1268.0], [1528.0, 1268.0], [1528.0, 1308.0], [1348.0, 1308.0]], ('Net price', 
0.9999516010284424)], [[[1580.0, 1268.0], [1780.0, 1268.0], [1780.0, 1308.0], [1580.0, 1308.0]], ('Net worth', 
0.9986754655838013)], [[[1842.0, 1268.0], [2001.0, 1268.0], [2001.0, 1308.0], [1842.0, 1308.0]], ('VAT [%]', 
0.9989665746688843)], [[[2134.0, 1261.0], [2259.0, 1261.0], [2259.0, 1312.0], [2134.0, 1312.0]], ('Gross', 
0.9985858798027039)], [[[2129.0, 1296.0], [2261.0, 1306.0], [2257.0, 1357.0], [2125.0, 1347.0]], (' worth', 
0.9336559772491455)], [[[351.0, 1388.0], [894.0, 1396.0], [893.0, 1444.0], [350.0, 1436.0]], ('Chindi Rugs Carpet 
New Design', 0.9996206164360046)], [[[1008.0, 1396.0], [1097.0, 1396.0], [1097.0, 1440.0], [1008.0, 1440.0]], 
('2,00', 0.9887767434120178)], [[[1418.0, 1392.0], [1532.0, 1392.0], [1532.0, 1443.0], [1418.0, 1443.0]], ('29,99',
0.9984349012374878)], [[[1672.0, 1392.0], [1787.0, 1392.0], [1787.0, 1443.0], [1672.0, 1443.0]], ('59,98', 
0.9979658126831055)], [[[1917.0, 1387.0], [2003.0, 1397.0], [1997.0, 1448.0], [1911.0, 1438.0]], ('10%', 
0.9998499751091003)], [[[1145.0, 1400.0], [1248.0, 1400.0], [1248.0, 1440.0], [1145.0, 1440.0]], (' each', 
0.913709819316864)], [[[2149.0, 1392.0], [2259.0, 1392.0], [2259.0, 1443.0], [2149.0, 1443.0]], ('65,98', 
0.9987150430679321)], [[[258.0, 1407.0], [277.0, 1407.0], [277.0, 1429.0], [258.0, 1429.0]], ('1', 
0.9433960914611816)], [[[354.0, 1436.0], [831.0, 1436.0], [831.0, 1484.0], [354.0, 1484.0]], ('Bohemian Garden Yoga
Mat', 0.99943321943283

In [ ]:
def chars_token_ratio(dataset, tokenizer, nb_examples=400):
    """
    Estimate the average number of characters per token in the dataset.
    """
    total_characters, total_tokens = 0, 0
    for _, example in tqdm(zip(range(nb_examples), iter(dataset)), total=nb_examples):
        text = format_train_instruction(example)
        total_characters += len(text)
        if tokenizer.is_fast:
            total_tokens += len(tokenizer(text).tokens())
        else:
            total_tokens += len(tokenizer.tokenize(text))

    return total_characters / total_tokens

In [ ]:
dataset_id="mychen76/invoices-and-receipts_ocr_v1"
data_dir="data/finetune"

def create_datasets(tokenizer,dataset_id,data_dir=None,seq_length=2048,num_workers=6,streaming=False,size_valid_set=10,shuffle_buffer=1000):
    dataset = load_dataset(
        dataset_id,
        data_dir=data_dir,
        split="train",
        num_proc=num_workers if not streaming else None,
        streaming=streaming,
    )
    if streaming:
        print("Loading the dataset in streaming mode")
        valid_data = dataset.take(size_valid_set)
        train_data = dataset.skip(size_valid_set)
        train_data = train_data.shuffle(buffer_size=shuffle_buffer, seed=None)
    else:
        dataset = dataset.train_test_split(test_size=0.003, seed=None)
        train_data = dataset["train"]
        valid_data = dataset["test"]
        print(f"Size of the train set: {len(train_data)}. Size of the validation set: {len(valid_data)}")

    chars_per_token = chars_token_ratio(train_data, tokenizer)
    print(f"The character to token ratio of the dataset is: {chars_per_token:.2f}")

    train_dataset = ConstantLengthDataset(
        tokenizer,
        train_data,
        formatting_func=format_train_instruction,
        infinite=True,
        seq_length=seq_length,
        chars_per_token=chars_per_token,
    )
    valid_dataset = ConstantLengthDataset(
        tokenizer,
        valid_data,
        formatting_func=format_train_instruction,
        infinite=False,
        seq_length=seq_length,
        chars_per_token=chars_per_token,
    )
    return train_dataset, valid_dataset

### Configuring Parameters



In [ ]:
################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = True

In [ ]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

In [ ]:
################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = True
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 2

# Batch size per GPU for evaluation
per_device_eval_batch_size = 2

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = False

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

In [ ]:
################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = 2048

# Pack multiple short examples in the same input sequence to increase efficiency
packing = True

In [ ]:
# Load base model
model_id = "microsoft/Phi-3-mini-4k-instruct"
device_string = PartialState().process_index

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map={'':device_string},
)

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/16.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

print_trainable_parameters(model)

trainable params: 197200896 || all params: 2009140224 || trainable%: 9.81518829021264

In [ ]:
model.eval()


Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear4bit(in_features=3072, out_features=9216, bias=False)
          (rotary_emb): Phi3RotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear4bit(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm()
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm()
      )
    )
    (norm): Phi3RMSNorm()
  )
  (lm_head): Linear(in_features=3072, 

In [ ]:
print(model.get_memory_footprint()/1024/1024/1024, "GB")
print(model.config.max_position_embeddings)
model.hf_device_map

2.054821014404297 GB

4096

{'': 0}

In [ ]:
# Load LoRA configuration
peft_config = LoraConfig(
        r=32,
        lora_alpha=8,
        target_modules=[
            "q_proj",
            "v_proj",
            "lm_head",
        ],
        bias="none",
        lora_dropout=0.05,
        task_type="CAUSAL_LM",
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)
model.config.use_cache = False
model.config.pretraining_tp = 1
model.print_trainable_parameters()

trainable params: 13,707,264 || all params: 2,022,847,488 || trainable%: 0.6776222172612946


In [ ]:
train_dataset, eval_dataset = create_datasets(tokenizer, dataset_id, seq_length=max_seq_length)

Size of the train set: 2036. Size of the validation set: 7

100%|██████████| 400/400 [00:06<00:00, 63.42it/s]


The character to token ratio of the dataset is: 1.35

In [ ]:
sft_config = SFTConfig(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard",
    packing=packing,  # Set packing here
    max_seq_length=max_seq_length  # Set max_seq_length here
)

trainer = SFTTrainer(
    model=model,
    peft_config=peft_config,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    args=sft_config  # Pass the SFTConfig object
)

### QLoRa Based SFT Training begin

In [ ]:
trainer.train()

You are not running the flash-attention implementation, expect numerical differences.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
25,0.948700
50,0.797100
75,0.752200
100,0.718600
125,0.698900
150,0.680400
175,0.671000
200,0.666300
225,0.653400
250,0.629500



TrainOutput(
    global_step=509,
    training_loss=0.6860241861849262,
    metrics={
        'train_runtime': 34766.3633,
        'train_samples_per_second': 0.059,
        'train_steps_per_second': 0.015,
        'total_flos': 9.347578330506854e+16,
        'train_loss': 0.6860241861849262,
        'epoch': 1.0
    }
)

In [ ]:
trainer.model.save_pretrained(new_model)
import gc
del model
del trainer
gc.collect()
torch.cuda.empty_cache()

### Merging and pushing to huggingface

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
model_id = "microsoft/Phi-3-mini-4k-instruct"
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
model = PeftModel.from_pretrained(base_model, "/kaggle/working/phi_miniocr_to_json")
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from huggingface_hub import login
login("token")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
model.push_to_hub("phi_miniocr_to_json", use_temp_dir=False)
tokenizer.push_to_hub("phi_miniocr_to_json", use_temp_dir=False)

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]


CommitInfo(
    commit_url='https://huggingface.co/elucidator8918/phi_miniocr_to_json/commit/d3603b85beb184b0448c5736f54dc55cf9c53cef',
    commit_message='Upload tokenizer',
    commit_description='',
    oid='d3603b85beb184b0448c5736f54dc55cf9c53cef',
    pr_url=None,
    pr_revision=None,
    pr_num=None
)